In [ ]:
import numpy as np
import menpo.io as mio
#import scipy.io as sio
from pathlib import Path
import os.path
import errno
#import menpo.shape
#%matplotlib inline
import re
from shutil import copyfile
import fnmatch
from PIL import Image

<b> Choose a directory</b>

In [ ]:
mat_contents = sio.loadmat('fatse3.mat')
pred=mat_contents.get('pred')

In [ ]:
# Function resize bounding box by %percent
def resize_bb(x,percent):
    width=x[3]-x[1]
    height=x[2]-x[0]
    print(percent)
    width_diff=percent*width
    height_diff=percent*height
    x[0]=x[0]-height_diff
    x[1]=x[1]-width_diff 
    x[2]=x[2]+height_diff
    x[3]=x[3]+width_diff

In [ ]:
num_not_detected_pic=0
num_not_exist_pic=0
num_cropped_pic=0
num_mult_detected_pic=0 

for indx in range(0,pred.shape[0]-1): 
    source_filename='../database/original/'+pred[indx][0]['name'][0][0][0] 
    dest_filename='../database/cropped_faces_larger_margin/'+pred[indx][0]['name'][0][0][0]
    if not os.path.exists(os.path.dirname(dest_filename)):
        try:
            os.makedirs(os.path.dirname(dest_filename))
        except OSError as exc: # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise
    source_file = Path(source_filename)
    
    if source_file.is_file():
        x=pred[indx][0]['box'][0][0].flatten()
        if (x.shape[0]==4):
            img=mio.import_image(source_filename)
            if x[0]<0:
                x[0]=0
                print(indx, source_filename,'Negative values found')
            if x[1]<0:
                x[1]=0
                print(indx, source_filename,'Negative values found')
            mio.export_image(img.crop([x[1],x[0]],[x[3],x[2]]),dest_filename,  extension=None,overwrite=True)
            num_cropped_pic=num_cropped_pic+1
        elif (x.shape[0]==0):
            print(indx, source_filename,' picture was not face detected')
            num_not_detected_pic=num_not_detected_pic+1
            dest_file = Path(dest_filename)
            if dest_file.is_file():
                os.remove(dest_filename)
                print(dest_filename,' was deleted from destination folder') 
        else: 
            num_faces=x.shape[0]//4
            print(indx, source_filename,' picture was multi-face detected. ',num_faces,' were detected')
            num_mult_detected_pic=num_mult_detected_pic+1
            img=mio.import_image(source_filename)
            parts = dest_filename.split('.jpg')
            for jj in range(1,num_faces+1):
                if x[4*(jj-1)]<0:
                    x[4*(jj-1)]=0
                    print(indx, source_filename,'Negative values found')
                if x[4*(jj-1)+1]<0:
                    x[4*(jj-1)+1]=0
                    print(indx, source_filename,'Negative values found')
                dest_filename=parts[0]+str(jj)+'.jpg'
                mio.export_image(img.crop([x[4*(jj-1)+1],x[4*(jj-1)]],[x[4*(jj-1)+3],x[4*(jj-1)+2]]),dest_filename,  extension=None,overwrite=True)
    else:
        print('File ',source_filename,' doesn''t exist')
        num_not_exist_pic=num_not_exist_pic+1
print('Num of single face decteted pic: ', num_cropped_pic )
print('\n Num of not decteted pic: ', num_not_detected_pic)
print('\n Num of not exist pic: ', num_not_exist_pic)
print('\n Num of multi face detected: ', num_mult_detected_pic)

In [ ]:
num_not_detected_pic=0
num_not_exist_pic=0
num_cropped_pic=0
num_mult_detected_pic=0 

perc_increase=0.4 


for indx in range(0,pred.shape[0]-1): 
    source_filename='../database/original/'+pred[indx][0]['name'][0][0][0] 
    dest_filename='../database/cropped2/'+pred[indx][0]['name'][0][0][0]
    check_filename='../database/cropped/'+pred[indx][0]['name'][0][0][0]
    if not os.path.exists(os.path.dirname(dest_filename)):
        try:
            os.makedirs(os.path.dirname(dest_filename))
        except OSError as exc: # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise
    source_file = Path(source_filename)
    
    if source_file.is_file():
        x=pred[indx][0]['box'][0][0].flatten()
        if (x.shape[0]==4):
#             Increase by perc_increase% width and height
            resize_bb(x,perc_increase)            
            img=mio.import_image(source_filename)
            if x[0]<0:
                x[0]=0
                print(indx, source_filename,'Negative values found')
            if x[1]<0:
                x[1]=0
                print(indx, source_filename,'Negative values found')
            mio.export_image(img.crop([x[1],x[0]],[x[3],x[2]]),dest_filename,  extension=None,overwrite=True)
            num_cropped_pic=num_cropped_pic+1
        elif (x.shape[0]==0):
            print(indx, source_filename,' picture was not face detected')
            num_not_detected_pic=num_not_detected_pic+1
            dest_file = Path(dest_filename)
            if dest_file.is_file():
                os.remove(dest_filename)
                print(dest_filename,' was deleted from destination folder') 
        else: 
            num_faces=x.shape[0]//4
            print(indx, source_filename,' picture was multi-face detected. ',num_faces,' were detected')
            num_mult_detected_pic=num_mult_detected_pic+1
            img=mio.import_image(source_filename)
            parts = dest_filename.split('.jpg')
#             parts2 = check_filename.split('.jpg')
            for jj in range(1,num_faces+1):
                if  Path(check_filename.split('.jpg')[0]+str(jj)+('.jpg')).is_file():
    #           Increase by perc_increase% width and height
                    resize_bb(x[4*(jj-1),4*(jj-1)+3],perc_increase)
#                     width=x[4*(jj-1)+3]-x[4*(jj-1)+1]
#                     height=x[4*(jj-1)+2]-x[4*(jj-1)]
#                     width_diff=perc_increase*width
#                     height_diff=perc_increase*height
#                     x[4*(jj-1)]=x[4*(jj-1)]-height_diff
#                     x[4*(jj-1)+1]=x[4*(jj-1)+1]-width_diff 
#                     x[4*(jj-1)+2]=x[4*(jj-1)+2]+height_diff
#                     x[4*(jj-1)+3]=x[4*(jj-1)+3]+width_diff

                    if x[4*(jj-1)]<0:
                        x[4*(jj-1)]=0
                        print(indx, source_filename,'Negative values found')
                    if x[4*(jj-1)+1]<0:
                        x[4*(jj-1)+1]=0
                        print(indx, source_filename,'Negative values found')
                    dest_filename=parts[0]+str(jj)+'.jpg'
                    mio.export_image(img.crop([x[4*(jj-1)+1],x[4*(jj-1)]],[x[4*(jj-1)+3],x[4*(jj-1)+2]]),dest_filename,  extension=None,overwrite=True)
    else:
        print('File ',source_filename,' doesn''t exist')
        num_not_exist_pic=num_not_exist_pic+1
print('Num of single face detected pic: ', num_cropped_pic )
print('\n Num of not detected pic: ', num_not_detected_pic)
print('\n Num of not exist pic: ', num_not_exist_pic)
print('\n Num of multi face detected: ', num_mult_detected_pic)

In [ ]:
print(check_filename.split('.jpg')[0]+str(jj)+('.jpg'))

In [ ]:
x=input()
print(x)

In [ ]:
indx=120
source_filename='../database/original/'+pred[indx][0]['name'][0][0][0]    
check_filename='../database/cropped/'+pred[indx][0]['name'][0][0][0]


parts = check_filename.split('.jpg')
jj=int(parts[0][-1])
print(jj)
print(check_filename)
print('Synthesized',parts[0][0:-1]+'1'+'.jpg')
print(source_filename)
img=mio.import_image(source_filename)
img.view()
x=pred[indx][0]['box'][0][0].flatten()
print(x.shape[0])
bb=menpo.shape.bounding_box([x[1],x[0]],[x[3],x[2]])
bb.view()
width=x[3]-x[1]
height=x[2]-x[0]
perc_increase=0.15 

width_diff=perc_increase*width
height_diff=perc_increase*height

bb3=menpo.shape.bounding_box([x[1]-width_diff,x[0]-height_diff],[x[3]+width_diff,x[2]+height_diff])
bb3.view()
bb2=menpo.shape.bounding_box([x[5],x[4]],[x[7],x[6]])
bb2.view()
print(x.shape[0]//2)
print(x.shape[0]%2)
parts = source_filename.split('.jpg')
print(parts)
source_filename2=parts[0]+str(1)+'.jpg'
print(source_filename2)

In [115]:
from time import time

In [119]:
e = time() 

In [124]:
time() - e

10.112269163131714

In [125]:
root = '11_06_2017/'
dest='tmp3'

num_images=0
num_obj=0
num_bmp=0
num_images_subj=[]
num_files=0
num_subj=1
num_frames_per_subj=[] 
min_ages=[]
max_ages=[]
names=[]
num_imgs=[]
temp_name=[]
names_files=[]
num_copied=0
# size= 1014, 1024 
size= 640, 480
ilevel=1

for path, dirs, files in os.walk(root):
    if ilevel==1:
        subjects=dirs
        print(path,dirs,files)        
    ilevel=ilevel+1
    for name in files:
        if fnmatch.fnmatch(name, '*.bmp'):
            num_images=num_images+1
    if 'meshes' in path.split('/')[-1]:
        for name in files:
            e = time()
            if fnmatch.fnmatch(name, '*.bmp'):
                
                tmp_name=name.split('/')
                new_name=tmp_name[-1].split('.')
                new_path=dest+'/'+new_name[0]
                if not os.path.exists(new_path):
                    os.makedirs(new_path)
                im = Image.open(path+'/'+name)
                im.thumbnail(size, Image.ANTIALIAS)
                
                im.save(new_path+'/'+new_name[0]+'.'+new_name[1]+'.jpg',quality=90)
                num_bmp=num_bmp+1
                print(time() - e)
            if fnmatch.fnmatch(name, '*.obj'):
                num_obj=num_obj+1
        num_frames_per_subj.append(num_obj)
        num_images_subj.append(num_bmp)
        num_obj=0
        num_bmp=0

print(len(subjects),ilevel,num_images,num_obj)

11_06_2017/ ['03696', '03717', '03738', '03676', '03677', '03678', '03679', '03680', '03681', '036811', '03682', '03683', '03684', '03685', '03686', '03687', '03688', '03689', '03690', '03691', '03692', '03693', '03694', '03695', '03697', '03698', '03699', '03700', '03701', '03702', '03703', '037031', '03704', '03705', '03706', '03707', '03708', '03709', '03710', '03711', '03712', '03713', '03714', '03715', '03716', '03719', '037191', '03720', '03721', '03722', '03723', '03724', '03725', '03726', '03727', '03728', '03729', '03730', '03731', '03732', '03733', '03734', '03735', '03736', '03737', '03739', '03740', '03741', '03742', '03744', '03745', '03746', '03747', '03748', '03749', '037491', '03750', '03751', '03752', '03753', '03754', '03755', '03756', '03757', '03758', '03759', '03760', '037601', '03761', '03762', '03763'] []
0.16422605514526367
0.14832758903503418
0.14598369598388672
0.14990234375
0.14862322807312012
0.15472674369812012
0.15114092826843262
0.15032458305358887
0.1494

KeyboardInterrupt: 

In [97]:
frame_id = im.path.stem

In [108]:
 path =  Path('tmp') / subject_id / im.path.with_suffix('.jpg').name

'03676'

In [3]:
import menpo.io as mio

from menpo.visualize import print_progress
from pathlib import Path

def process_image(im):
    im = im.rescale_to_diagonal(700)
    subject_id = im.path.stem.split('.')[0]
    path =  Path('tmp') / subject_id / im.path.with_suffix('.jpg').name
    
    try:
        mio.export_image(im, path, overwrite=True)
    except:
        path.parent.mkdir(exist_ok=True, parents=True)
        mio.export_image(im, path)

# for im in print_progress(mio.import_images('/media/ap112/Seagate Expansion Drive/11_06_2017/**/meshes/*.bmp')):


In [5]:
from multiprocessing import Pool

pool = Pool(4)

pool.map_async(process_image, mio.import_images('/media/ap112/Seagate Expansion Drive/11_06_2017/**/meshes/*.bmp'))

In [4]:
pool.map_async()

In [ ]:
unique_names, unique_counts = np.unique(names3,return_counts=True)
# unique_names=np.array(unique_names) 
# unique_counts=np.array(unique_counts)
duplicate_names= unique_names[unique_counts>1]
# names2==duplicate_names[0]
# names.index(unique_names[0])
# indexes=np.where(names3==duplicate_names[0])
# print(duplicate_names)
# print(unique_counts[unique_counts>1])
print(len(unique_names),names3.shape)
print(names3)